In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.Splendor_v3.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((200000,getStateSize())),np.zeros((200000,1)),np.zeros((200000,1)),np.zeros((1,1)),np.zeros((1,1))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11           #12

In [3]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((200000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((200000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the value the lowest average reward
    min = 0
    index = 0
    for i in range(len(count)):
        if count[i][2] != 0:
            min = count[i][2]/count[i][1]
            index = i
            break
    return count[index][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0


In [4]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 1000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 1000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 200000 :
      per[5][0][0] = 4
      per[2][0][0] = 0
      per[4][0][0] = 0
      per[4][0][1] = 0 
      per[5][0][1] = 0 

    # Bất đầu mode train và test policy yếu hơn
    if per[11][0][0] == 200000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 1000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 1000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 10000 and per[11][0][0] < 200000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 10000 and per[11][0][0] == 200000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
    if per[11][0][0] == 200000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 10000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <1000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 10000 and per[5][0][0] == 2 and per[11][0][0] == 200000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and Check_yeu == 1:
        per[1] += per[0][0]
      if getReward(state) == 0 and Check_yeu == 1:
        per[0][0] = np.random.rand(getActionSize())

    # chế độ test policy yếu
    elif int(per[2][0][0]) <1000 and  per[5][0][0] == 3 and per[11][0][0] == 200000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  


    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 200000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 200000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 200000,perx,0)

Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu
Policy yếu

In [ ]:
perx

[array([[0.91677339, 0.97849425, 0.5395696 , 0.16583203, 0.699038  ,
         0.08285082, 0.77957335, 0.00963059, 0.3528644 , 0.08876591,
         0.79897086, 0.39039909, 0.70397447, 0.02008768, 0.58443073]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[6938.]]),
 array([[0.91691135, 0.94188113, 0.96167511, 1.        , 0.88987479,
         0.92469833, 0.8993115 , 0.90130476, 0.90688674, 0.89429272,
         0.90950004, 0.92165595, 0.91064024, 0.96440533, 0.92811731]]),
 array([[0., 0.]]),
 array([[  2., 614.]]),
 array([[3.]]),
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan]]),
 array([[ 7.,  3.,  7., ...,  1., 12.,  0.],
        [ 7.,  2.,  6., ...,  1., 12.,  0.],
        [ 6.,  0.,  4., ...,  1., 12.,  0.],
        ...,
        [ 0.,  1.,  0., ...,  1.,  8.,  0.],
        [ 0.,  0.,  2., ...,  1.,  8.,  0.],
        [ 3.,  1.,  3., ...,  1.,  8.,  0.]]),
 array([[3.],
        [3.],
        [3.],
      

# Find the most important value of index in the state that when it appear the policy dont work well


In [ ]:
def find_most_importane_feature(perx):
    import pandas as pd

    df = pd.DataFrame(perx[8])
    df['action'] = perx[9]
    df['reward'] = perx[10]
    for i in range(len(df)):
        if df['reward'][i] == -1:
            for j in range(i,len(df)):
                if df['reward'][j] != -1:
                    df['reward'][i] = df['reward'][j]
                    break
    df = df[df['reward'] != -1]

    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier

    # Load the state-action dataset

    # Split the data into features (state properties) and target (outcome)
    features = df.drop("reward", axis=1)
    target = df["reward"]

    # Train a Random Forest Classifier on the data
    clf = RandomForestClassifier()
    clf.fit(features, target)

    # Get the feature importances
    importances = clf.feature_importances_

    # Find the property (x) of the state that is most important for the policy's performance
    x = features.columns[importances.argmax()]
    # Plot the top  feature importances 
    import matplotlib.pyplot as plt
    import numpy as np
    indices = np.argsort(importances)[::-1]
    plt.figure(figsize=(10, 5))
    plt.title("Feature importances")
    plt.bar(range(features.shape[1]), importances[indices], color="r", align="center")
    plt.xticks(range(features.shape[1]), features.columns[indices], rotation=90)
    plt.xlim([-1, features.shape[1]])
    plt.show()
    most_imp_feature = None
    lowest_percentage = 100
    print('feature importances',x)
    for i in df[x].unique():
        current_percentage = df[df[x] == i]['reward'].value_counts()[1]/df[df[x] == i]['reward'].value_counts().sum()
        if current_percentage < lowest_percentage:
            most_imp_feature = i
            lowest_percentage = current_percentage

    print("The value of the most important feature with the lowest percentage of value 1 in reward is:", most_imp_feature)#  all the below code in one function    
    return most_imp_feature
